## Distill SFT



```bash
# unsloth 有个bug，会不认 Huggingface 的 model cache, 所以需要手动下载
huggingface-cli download --resume-download Qwen/Qwen2.5-1.5B-Instruct --local-dir ~/models/Qwen2.5-1.5B-Instruct
```

In [1]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch

max_seq_length = 8192 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/home/cynic/models/Qwen2.5-1.5B-Instruct", # change to your model path
    max_seq_length = max_seq_length,
    load_in_4bit = False, # False for LoRA 16bit
    local_files_only=True,
    #fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    #gpu_memory_utilization = 0.5, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank * 2,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-20 10:49:05 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Qwen2 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading /home/cynic/models/Qwen2.5-1.5B-Instruct with actual GPU utilization = 46.74%
Unsloth: Your GPU has CUDA compute capability 8.9 with VRAM = 23.99 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-20 10:49:18 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 04-20 10:49:18 [model_runner.py:1146] Model loading took 1.2131 GiB and 2.050700 seconds
INFO 04-20 10:49:20 [worker.py:267] Memory profiling takes 1.25 seconds
INFO 04-20 10:49:20 [worker.py:267] the current vLLM instance can use total_gpu_memory (23.99GiB) x gpu_memory_utilization (0.47) = 11.21GiB
INFO 04-20 10:49:20 [worker.py:267] model weights take 1.21GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.23GiB; the rest of the memory reserved for KV Cache is 8.69GiB.
INFO 04-20 10:49:20 [executor_base.py:112] # cuda blocks: 20347, # CPU blocks: 14043
INFO 04-20 10:49:20 [executor_base.py:117] Maximum concurrency for 2048 tokens per request: 158.96x
INFO 04-20 10:49:21 [model_runner.py:1456] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. 

Capturing CUDA graph shapes:   0%|          | 0/31 [00:00<?, ?it/s]

INFO 04-20 10:49:39 [model_runner.py:1598] Graph capturing finished in 18 secs, took 0.54 GiB
INFO 04-20 10:49:39 [llm_engine.py:449] init engine (profile, create kv cache, warmup model) took 20.67 seconds


Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.
Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [4]:
# 可以使用训练中定义的，也可以使用模型自带的 chat template
#from unsloth.chat_templates import get_chat_template
#tokenizer = get_chat_template(
#   tokenizer,
#  chat_template = "qwen-2.5",
#)

In [2]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="data/train_sft_distill.jsonl")["train"]

In [3]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["messages"])
    texts = [tokenizer.decode(text) for text in texts]
    return { "text" : texts }

dataset = dataset.map(apply_chat_template, batched = True, load_from_cache_file=False)

Map:   0%|          | 0/712 [00:00<?, ? examples/s]

In [4]:
print(dataset[99]["text"])

<|im_start|>system
You are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer.<|im_end|>
<|im_start|>user
Using the numbers 30, 24, 10, 35, create an equation that equals 43. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once, and you must use all the numbers. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer>(1 + 2) / 3</answer>. Think step by step inside <think> tags.<|im_end|>
<|im_start|>assistant
<think>Okay, let's see. I need to make an equation using all the numbers 30, 24, 10, and 35 exactly once, combining them with basic arithmetic operations (+, -, *, /) to get 43. Hmm, let's start by thinking about possible combinations.

First, 43 is a prime number, so maybe addition or subtraction could be part of the solution. Let's look at the numbers: 30, 24, 10, 35. The largest number is

In [5]:
import wandb
wandb.init(project="countdown-sft-distill")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: swulling to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        max_seq_length = max_seq_length, # 最大长度
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 1e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,
        report_to = "wandb", # Use this for WandB etc
    ),
)

In [7]:
print(tokenizer.decode(trainer.train_dataset[99]["input_ids"]))

<|im_start|>system
You are a helpful assistant. You first thinks about the reasoning process in the mind and then provides the user with the answer.<|im_end|>
<|im_start|>user
Using the numbers 30, 24, 10, 35, create an equation that equals 43. You can use basic arithmetic operations (+, -, *, /) one or multiple times but each number can only be used once, and you must use all the numbers. Show your work in <think> </think> tags. And return the final equation in <answer> </answer> tags, for example <answer>(1 + 2) / 3</answer>. Think step by step inside <think> tags.<|im_end|>
<|im_start|>assistant
<think>Okay, let's see. I need to make an equation using all the numbers 30, 24, 10, and 35 exactly once, combining them with basic arithmetic operations (+, -, *, /) to get 43. Hmm, let's start by thinking about possible combinations.

First, 43 is a prime number, so maybe addition or subtraction could be part of the solution. Let's look at the numbers: 30, 24, 10, 35. The largest number is

In [ ]:
# todo: train completions only

In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 712 | Num Epochs = 3 | Total steps = 267
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 73,859,072/1,617,573,376 (4.57% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.271500
2,1.236200
3,1.290200
4,1.268600
5,1.316600
6,1.203700
7,1.191800
8,1.163500
9,1.132800
10,1.045400


wandb link: https://wandb.ai/swulling/countdown-sft-distill/runs/aaenr5bf?nw=nwuserswulling

In [9]:
model.save_pretrained("output/qwen2.5-1.5b-sft-distill-lora")  # Local saving lora weights
tokenizer.save_pretrained("output/qwen2.5-1.5b-sft-distill-lora")

('output/qwen2.5-1.5b-sft-distill-lora/tokenizer_config.json',
 'output/qwen2.5-1.5b-sft-distill-lora/special_tokens_map.json',
 'output/qwen2.5-1.5b-sft-distill-lora/vocab.json',
 'output/qwen2.5-1.5b-sft-distill-lora/merges.txt',
 'output/qwen2.5-1.5b-sft-distill-lora/added_tokens.json',
 'output/qwen2.5-1.5b-sft-distill-lora/tokenizer.json')

vllm inference
```bash
vllm serve Qwen/Qwen2.5-1.5B-Instruct --port 8100 --api-key NLUKKXIJDZ91rpg1z --enforce-eager  --max-model-len 10000 --enable-lora --max-lora-rank 64 --lora-modules qwen2.5-1.5b-sft-distill-lora=output/qwen2.5-1.5b-sft-distill-lora

CURATOR_VIEWER=1 python eval.py --provider vllm --data_path data/test.jsonl --model_name qwen2.5-1.5b-sft-distill-lora --temperature 0.01 --max_tokens 8192

https://curator.bespokelabs.ai/datasets/38a2fef9c1614b0089c12ea38d2248e6    

Accuracy: 44/100 (44.00%)
```

分析：

1. 格式上 SFT 后完全遵守，没有问题
2. 相比于简单的 SFT，因为增加大量思考过程，还是会出现数字使用错误（包括没有使用或者使用超过一次）
3. 正确率有显著提升，证明模型确实学习到了推理过程
4. 大量错误都是因为无法找到正确答案超过最长 tokens 限制。
